# Amazon SageMaker Semantic Segmentation Algorithm - v9: deploy

post migration from [roofAI](https://github.com/kamangir/roofAI/tree/main/notebooks/sagemaker)

In [1]:
# !pip3 install 'sagemaker>=2,<3'

In [2]:
import sys
sys.path.append("../../")

from blueflow import notebooks

import sagemaker

from blue_options import string

from blue_sandbox import fullname
from blue_sandbox.env import SAGESEMSEG_COMPLETED_JOB
from blue_sandbox.logger import logger

logger.info(f"{fullname()}, built on {string.pretty_date()}")

/Users/kamangir/git/.venv/lib/python3.9/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config      - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config      - Not applying SDK defaults from location: /Users/kamangir/Library/Application Support/sagemaker/config.yaml


🌀  blue_sandbox-5.109.1, built on 12 January 2025, 14:02:48


In [3]:
# attach to a completed training job
ss_estimator = sagemaker.estimator.Estimator.attach(SAGESEMSEG_COMPLETED_JOB)


2025-01-12 06:05:43 Starting - Preparing the instances for training
2025-01-12 06:05:43 Downloading - Downloading the training image
2025-01-12 06:05:43 Training - Training image download completed. Training in progress.
2025-01-12 06:05:43 Uploading - Uploading generated training model
2025-01-12 06:05:43 Completed - Training job completed


## Deployment

Once the training is done, we can deploy the trained model as an Amazon SageMaker hosted endpoint. This will allow us to make predictions (or inference) from the model.

Note that we don't have to host on the same number or type of instances that we used to train, and can choose any SageMaker-supported instance type. Training is compute-heavy job that may have different infrastructure requirements than inference/hosting. In our case we chose the GPU-accelerated `ml.p3.2xlarge` instance to train, but will host the model on a lower cost-per-hour `ml.c5.xlarge` type - because we'll only be serving occasional requests.

The endpoint deployment can be accomplished as follows:

In [ ]:
ss_predictor = ss_estimator.deploy(initial_instance_count=1, instance_type="ml.c5.xlarge")

sagemaker Creating model with name: sagesemseg-model-2025-01-11-22-00-08-bW-2025-01-12-22-02-54-736
sagemaker Creating endpoint-config with name sagesemseg-model-2025-01-11-22-00-08-bW-2025-01-12-22-02-54-736
sagemaker Creating endpoint with name sagesemseg-model-2025-01-11-22-00-08-bW-2025-01-12-22-02-54-736


----

In [ ]:
# As with Estimators & training jobs, we can instead attach to an existing Endpoint:
# ss_predictor = sagemaker.predictor.Predictor("ss-notebook-demo-2020-10-29-07-23-03-086")

## Inference

Now that the trained model is deployed to an endpoint, we can use this endpoint for inference.

To test it out, let us download an image from the web which the algorithm has so-far not seen. 

In [ ]:
filename_raw = "data/test.jpg"
!mkdir "data"

!wget -O $filename_raw https://upload.wikimedia.org/wikipedia/commons/b/b4/R1200RT_in_Hongkong.jpg

In [ ]:
from matplotlib import pyplot as plt
import PIL

%matplotlib inline

filename = "data/test_resized.jpg"
width = 800

im = PIL.Image.open(filename_raw)

aspect = im.size[0] / im.size[1]

im.thumbnail([width, int(width / aspect)], PIL.Image.LANCZOS)
im.save(filename, "JPEG")

plt.imshow(im)
plt.show()

In [ ]:
from PIL import Image
import numpy as np


class ImageDeserializer(sagemaker.deserializers.BaseDeserializer):
    """Deserialize a PIL-compatible stream of Image bytes into a numpy pixel array"""

    def __init__(self, accept="image/png"):
        self.accept = accept

    @property
    def ACCEPT(self):
        return (self.accept,)

    def deserialize(self, stream, content_type):
        """Read a stream of bytes returned from an inference endpoint.
        Args:
            stream (botocore.response.StreamingBody): A stream of bytes.
            content_type (str): The MIME type of the data.
        Returns:
            mask: The numpy array of class labels per pixel
        """
        try:
            return np.array(Image.open(stream))
        finally:
            stream.close()


ss_predictor.deserializer = ImageDeserializer(accept="image/png")

In [ ]:
ss_predictor.serializer = sagemaker.serializers.IdentitySerializer("image/jpeg")

with open(filename, "rb") as imfile:
    imbytes = imfile.read()

# Extension exercise: Could you write a custom serializer which takes a filename as input instead?

In [ ]:
%%time

cls_mask = ss_predictor.predict(imbytes)

print(type(cls_mask))
print(cls_mask.shape)

In [ ]:
plt.imshow(cls_mask, cmap="jet")
plt.show()

In [ ]:
ss_predictor.delete_endpoint()